In [ ]:
import os, from __future__ import print_function, division, import cv2
import numpy as np, from tqdm import tqdm
import torch, import torch.nn.functional as F, import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, from torchvision.transforms import transforms, from torch.utils.data import DataLoader, from torch.utils.data import Dataset
import pandas as pd
from sklearn import preprocessing
import torch.nn as nn


mutated_data = pd.read_csv('mutated_data - Sheet1.csv').sample(frac=1)
load_data = mutated_data.drop('Disease class (1=healthy, 2=cf, 3=gt, 4=bs, 5=sc)', axis=1)


load_data = (load_data-load_data.min())/(load_data.max()-load_data.min()) ##normalized data
load_data = torch.tensor(pd.DataFrame(data=load_data).values)

load_labels = mutated_data.iloc[:, -1] #labels of disease

labels = []

for item in load_labels: 
  disc_class = [0, 0, 0, 0, 0]
  index = 0
  disc_class[int(item)-1] = 1
  labels.append(disc_class)
  index += 1

load_labels = torch.FloatTensor(labels)
train_data = []

for i in range(len(load_data)):
   train_data.append([load_data[i], load_labels[i]])

training = train_data[0:int(len(train_data) * .80)]
testing = train_data[(int(len(train_data) * .80)): (int(len(train_data) * 1))]

trainloader = torch.utils.data.DataLoader(training, shuffle=True, batch_size=50)
testloader = torch.utils.data.DataLoader(testing, shuffle=True, batch_size=50)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(26, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 5)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)
        
        return x
    
net = Net()


import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=1e-3)

EPOCHS = 5

for epoch in range(EPOCHS):
    for data in trainloader:
        X, y = data
        net.zero_grad()
        output = net(X.float())
        y.view(-1, 5)
    #     loss = F.nll_loss(output, F.log_softmax(y))
    #     loss.backward()
    #     optimizer.step()
    # print(loss)




